In [1]:
import sys

if ".." not in sys.path:
    sys.path.insert(0, "..")

In [2]:
from copy import deepcopy
from datetime import datetime

from catppuccin.extras.rich_ctp import mocha
from rich import pretty
from rich.console import Console
from rich.pretty import pprint

from src.config.settings import get_settings
from src.utils.logger import ChatPrinter, create_logger

console = Console(theme=mocha)
settings = get_settings()
logger = create_logger(path=settings.paths.logs_dir)
printer = ChatPrinter()
logger.debug(f"settings loaded as \n{settings.model_dump_json(indent=2)}")

pretty.install()


11:14 AM | 2341847716.L:16  | DEBUG     | settings loaded as 
{
  "models": {
    "hf": {
      "chat": "meta-llama/Llama-3.2-3B-Instruct:together",
      "reasoning": "zai-org/GLM-4.7-Flash:novita",
      "structured_output": "Qwen/Qwen3-Coder-30B-A3B-Instruct:ovhcloud",
      "embedding_snowflake": "Snowflake/snowflake-arctic-embed-l-v2.0",
      "embedding_specter": "allenai/specter2_base",
      "encoder": "m3rg-iitd/matscibert",
      "reranker": "Qwen/Qwen3-Reranker-0.6B",
      "router": "openai/gpt-oss-20b:together"
    },
    "nebius": {
      "reasoning": "deepseek-ai/DeepSeek-V3.2",
      "tool_user": "Qwen/Qwen3-30B-A3B-Instruct-2507",
      "chat": "meta-llama/Meta-Llama-3.1-8B-Instruct-fast",
      "embedding_baai_bge": "BAAI/bge-multilingual-gemma2",
      "router": "openai/gpt-oss-20b"
    }
  },
  "paths": {
    "base_dir": "/home/rudy/code/lattice/src",
    "data_dir": "/home/rudy/code/lattice/src/data",
    "logs_dir": "/home/rudy/code/lattice/src/logs",
    "agents_

In [3]:
from typing import Annotated, Dict, List, Literal, Optional, TypedDict

from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    HumanMessage,
    MessageLikeRepresentation,
    SystemMessage,
    ToolMessage,
    convert_to_openai_messages,
    filter_messages,
    get_buffer_string,
)
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import StructuredTool, tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_nebius import ChatNebius
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.runtime import Runtime
from langgraph.types import Command, Overwrite, interrupt
from pydantic import BaseModel, Field, create_model
from pydantic.json_schema import SkipJsonSchema

In [4]:
from src.backends.filesystem import VirtualFilesystem
from src.prompts import (
    ask_mode_system_prompt,
    planning_mode_system_prompt,
    planning_structured_output,
)
from src.schemas import (
    RESEARCH_PLAN_TEMPLATE,
    ResearchPlan,
    SystemPromptTemplate,
)
from src.tools.filesystem import create_filesystem_tools
from src.tools.thinking import (
    switch_to_ask_mode_tool,
    switch_to_execution_mode_tool,
    switch_to_planning_mode_tool,
    think_tool,
)
from src.tools.utils import SkipSchema, wrap_tool_with_doc_and_error_handling
from src.utils.stats import accumulate_usage, add_usage_metadata

version = "0.0.1-alpha"

In [5]:
from deepagents.middleware import FilesystemMiddleware, SummarizationMiddleware
from deepagents.backends import StateBackend, FilesystemBackend, CompositeBackend
from deepagents import create_deep_agent

In [ ]:
backend = lambda rt: CompositeBackend(
    default=StateBackend(rt),
    routes={
        # treat filesystem as object and kv storage right now
        # vfs proxies
        "/notes/": StateBackend(rt),
        # object store proxies
        "/memories/": FilesystemBackend(
            root_dir=settings.paths.agentfs_dir / "memories", virtual_mode=True
        ),
        "/artifacts/": FilesystemBackend(
            root_dir=settings.paths.agentfs_dir / "artifacts", virtual_mode=True
        ),
    },
)


In [14]:
filesystem_mw = FilesystemMiddleware(backend=backend)

In [17]:
filesystem_mw = FilesystemMiddleware(backend=backend)
filesystem_mw.tools[0].args_schema = wrap_tool_with_doc_and_error_handling(
    filesystem_mw.tools[0].func,
    custom_name=filesystem_mw.tools[0].name,
    custom_description=filesystem_mw.tools[0].description,
    custom_param_descriptions={
        "path": "Path to directory or file. If directory, list of info for files in this directory is returned. If file, info for file is returned."
    },
).args_schema

In [8]:
from langchain.agents import create_agent